# Frequent itemset mining

- Student ID: 21127166
- Student name: Nguyen Tuan Thanh

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [10]:
from collections import defaultdict

### Read data
First we have to read data from database

In [11]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) #* Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1
        for line in f:
            itemset=set(map(int,line.split())) #* A python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #* Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    return data, s

### Tree Projection

**Question 0: I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**Answer:**

In [12]:
def joinset(a,b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space) 

    -------------------
    return 
        ret: itemset generated by joining a and b
    '''
    # a,b same brach -> a_k = b_k
    # Hint: this function will be called in generateSearchSpace method.:
    return list(set(a).union(b))
    
    
class TP:
    def __init__ (self, data=None, s=None, minSup=None):
        self.data=data
        self.s={}
        
        
        for key, support in sorted(s.items(),key= lambda item: item[1]):
            self.s[key]=support
        # Why should we do this, answer it at the markdown below?
            
        self.minSup=minSup
        self.L={}  #Store frequent itemsets mined from database
        self.runAlgorithm()
        
    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree={}

        search_space={}  
        for item, support in self.s.items():
            search_space[item]={}
            
            search_space[item]['itemset']=[item] 
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.
            
            search_space[item]['pruned']=False

            
            search_space[item]['support']=support
            
            tree[item]={}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.
            
            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            ''' 
        
        return tree, search_space
    
    def computeItemsetSupport(self, itemset):
        '''Return support of itemset'''
        itemset = set(itemset)  # Ensure itemset is a set
        return sum(itemset.issubset(itemsets) for _, itemsets in self.data.items())
    
    
        
    def prune(self,k, tree, search_space):  
        '''
        In this method we will find out which itemset in current search space is frequent 
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets. 
        '''
        #TODO
        items = list(tree.keys())
        for item in items:
            if search_space[item]['support'] >= self.minSup:
                self.L[k] = self.L.get(k, []) + [search_space[item]['itemset']]
            else:
                search_space[item]['pruned'] = True
       
    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function) 
        '''
        items=list(tree.keys())  
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l=len(items)
        self.prune(k, tree, search_space)
        if l==0: return   #Stop condition
        
        for i in range(l-1):
            a=items[i]
            if search_space[a]['pruned']: continue 
                
            for j in range(i+1,l):
                b=items[j]
                
                search_space[a][b]={}
                tree[a][b]={}
                #TODO
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft). 
            
                #First create newset using join set
                newset = joinset(search_space[a]['itemset'], search_space[b]['itemset'])
                
                #Second add newset to search_space
                search_space[a][b]['itemset'] = newset
                search_space[a][b]['pruned'] = False
                search_space[a][b]['support'] = self.computeItemsetSupport(newset)
                
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k+1,tree[a],search_space[a])

    
    def runAlgorithm(self):
        tree, search_space=self.initialize() #generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)
        
    def miningResults(self):
        return self.L
        

Ok, let's test on a typical dataset `chess`.

In [13]:
transactions, freq= readData('chess.txt')

In [14]:
# Run and print result (better print format)
a=TP(data=transactions,s=freq, minSup=3000)
print(*a.miningResults().items(),sep="\n")

(1, [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]])
(2, [[48, 52], [48, 58], [56, 29], [56, 52], [56, 58], [66, 60], [66, 29], [66, 52], [66, 58], [40, 34], [34, 29], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [60, 7], [40, 7], [29, 7], [52, 7], [58, 7], [36, 60], [40, 36], [36, 29], [36, 52], [58, 36], [40, 60], [60, 29], [60, 52], [58, 60], [40, 29], [40, 52], [40, 58], [52, 29], [58, 29], [58, 52]])
(3, [[48, 58, 52], [56, 52, 29], [56, 58, 29], [56, 58, 52], [66, 60, 29], [66, 60, 52], [66, 60, 58], [66, 52, 29], [66, 58, 29], [66, 52, 58], [40, 34, 29], [40, 34, 52], [40, 34, 58], [34, 52, 29], [34, 58, 29], [34, 52, 58], [60, 29, 62], [60, 62, 52], [58, 60, 62], [40, 29, 62], [40, 52, 62], [40, 58, 62], [52, 29, 62], [58, 29, 62], [58, 52, 62], [40, 60, 7], [60, 29, 7], [60, 52, 7], [58, 60, 7], [40, 29, 7], [40, 52, 7], [40, 58, 7], [52, 29, 7], [58, 29, 7], [58, 52, 7], [40, 36, 60], [36, 29, 60], [36, 60, 52], [58, 36, 60], [

### Answer questions here:


**Question 1. Why should we sort all items in s by it's support and why we have to restored it to new artribute s?**

**Answer:**

We sort all items in s by its support to prune early those nodes that do not meet min_sup first without generating child branches from that node. 

For example, if we do not sort items by support, suppose we are considering node A that satisfies min_sup and there exists a node B behind node A, then according to the algorithm, we will still create the branch A->B even though this branch does not meet min_sup and this branch is only pruned when we consider the next level of node A. 

Therefore, we sort items by support will eliminate this case without creating a new branch and thus directly improve the performance of the algorithm. We store the result in a different variable s to make it easy to sort and initialize in the order of support we want in the initialize step.


**Question 2. Python set does not remain elements order so we use a list to extend it easily when create new itemset but why we store itemset in data by a python set????**

**Answer:**
We store itemsets in data by a Python set because sets in Python have many advantages that are better suited for this lab.

+ Eliminating Duplicates: Sets ensure that no element appears multiple times, eliminating redundancy in the representation of itemsets. This is particularly important in frequent itemset mining, where the focus is on identifying unique combinations of items.

+ Efficient Set Operations: Sets provide efficient operations for common set-related tasks, such as checking membership, finding unions, intersections, and differences. These operations are crucial for algorithms that analyze frequent itemsets.


**Question 3.  After finish implementing the algorithm tell me why should you use this instead of delete item directly from search_space and tree.**

**Answer:**

We should use a pruning function to remove itemsets from the tree and search_space instead of deleting them directly. This is because deleting itemsets directly can lead to the loss of useful information.

For example, suppose we have the following itemsets in the tree: {A, B} - {A, C}
If we delete the itemset {A, B}, the node for {A, B} will no longer be in the tree. However, the node for {A, C} is still in the tree. If we then try to generate the itemset {A, B, C}, we will not be able to find the node for {A, B}.


# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). You can download dataset from here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/churn.txt. Write your report and implementation below.

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  
More information about `churn` here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir4/Churn.pdf

**REPORT**

In the first step, I used the processedData function to process the churn.txt file into a format that is compatible with the chess.txt file. This involved splitting the header into individual features and assigning them to the corresponding data in each column. For example, the State attribute with a value of KS would be changed to [State]_KS. The reason for processing the header is to ensure that the algorithm can correctly identify the attributes of each transaction. This is important because the algorithm uses the attributes to determine whether an itemset is frequent.

In the second step, I used the min support algorithm to find frequent itemsets. The min support value was set to 1/2 of the number of transactions. This means that an itemset must appear in at least half of the transactions in order to be considered frequent. 

**TODO:**

In [15]:
from collections import defaultdict

def writeData(data, file_path):
    """
    Writes the given data to a file.

    Parameters:
        data: A dictionary where the keys are transaction ids and the values are itemsets.
        file_path: The path to the file to write to.
    """
    with open(file_path, 'wt') as f:
        f.writelines(','.join(itemset) + '\n' for itemset in data.values())

def processedData(file_path):
    """
    Processes the data from a file and writes it to another file.

    Parameters:
        file_path: The path to the file to read from.
    """
    with open(file_path, 'rt') as f:
        headers = next(f).strip().split(',')
        data = {tid: [f'[{header}]_{item}' for header, item in zip(headers, line.strip().split(','))]
                for tid, line in enumerate(f, start=1)}
    writeData(data, 'processed.txt')

def readData(file_path):
    """
    Reads data from a file and returns a dictionary representing the database and a dictionary of item supports.

    Parameters:
        file_path: The path to the file to read from.

    Returns:
        data: A dictionary representing the database where the keys are transaction ids and the values are itemsets.
        s: A dictionary where the keys are items and the values are their supports.
    """
    data = {}
    s = defaultdict(int)
    with open(file_path, 'rt') as f:
        for tid, line in enumerate(f, start=1):
            itemset = set(line.strip().split(','))
            data[tid] = itemset
            for item in itemset:
                s[item] += 1
    return data, s

In [16]:
processedData('churn.txt')

In [17]:
transactions, freq = readData('processed.txt')
print(transactions[1])
print(freq)

{'[VMail Plan]_yes', '[Intl Mins]_10.000000', '[Eve Calls]_99', '[Day Mins]_265.100000', '[Area Code]_415', '[Day Charge]_45.070000', '[Night Mins]_244.700000', '[Phone]_382-4657', '[CustServ Calls]_1', '[Intl Calls]_3', '[State]_KS', '[Eve Mins]_197.400000', '[Intl Charge]_2.700000', '[Night Calls]_91', '[Night Charge]_11.010000', '[Eve Charge]_16.780000', '[Account Length]_128', '[Day Calls]_110', '[Churn?]_False.', '[VMail Message]_25', "[Int'l Plan]_no"}
defaultdict(<class 'int'>, {'[VMail Plan]_yes': 922, '[Intl Mins]_10.000000': 62, '[Eve Calls]_99': 59, '[Day Mins]_265.100000': 1, '[Area Code]_415': 1655, '[Day Charge]_45.070000': 1, '[Night Mins]_244.700000': 4, '[Phone]_382-4657': 1, '[CustServ Calls]_1': 1181, '[Intl Calls]_3': 668, '[State]_KS': 70, '[Eve Mins]_197.400000': 5, '[Intl Charge]_2.700000': 62, '[Night Calls]_91': 76, '[Night Charge]_11.010000': 5, '[Eve Charge]_16.780000': 5, '[Account Length]_128': 22, '[Day Calls]_110': 66, '[Churn?]_False.': 2850, '[VMail Mes

In [18]:
b=TP(data=transactions, s=freq, minSup=0.5*len(transactions))
print(*b.miningResults().items(), sep="\n")

(1, [['[VMail Message]_0'], ['[VMail Plan]_no'], ['[Churn?]_False.'], ["[Int'l Plan]_no"]])
(2, [['[VMail Plan]_no', '[VMail Message]_0'], ['[Churn?]_False.', '[VMail Message]_0'], ['[VMail Message]_0', "[Int'l Plan]_no"], ['[VMail Plan]_no', '[Churn?]_False.'], ['[VMail Plan]_no', "[Int'l Plan]_no"], ['[Churn?]_False.', "[Int'l Plan]_no"]])
(3, [['[VMail Plan]_no', '[Churn?]_False.', '[VMail Message]_0'], ['[VMail Plan]_no', '[VMail Message]_0', "[Int'l Plan]_no"], ['[Churn?]_False.', '[VMail Message]_0', "[Int'l Plan]_no"], ['[VMail Plan]_no', '[Churn?]_False.', "[Int'l Plan]_no"]])
(4, [['[VMail Plan]_no', '[Churn?]_False.', '[VMail Message]_0', "[Int'l Plan]_no"]])


# 4 References

http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/ProjectDefinition1.pdf

https://cs.wmich.edu/~alfuqaha/summer14/cs6530/lectures/AssociationAnalysis-Part1.pdf

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
